In [2]:
import requests

# Specify the URL of the page you want to download
url = "https://pokemondb.net/pokedex/all"

# Make a GET request to fetch the raw HTML content
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Save the content to an HTML file
    with open("pokemon.html", "w", encoding="utf-8") as file:
        file.write(response.text)
    print("Page saved as 'page.html'")
else:
    print(f"Failed to download page. Status code: {response.status_code}")

Page saved as 'page.html'


In [4]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

#URL der Webseite
url = "https://pokemondb.net/pokedex/all"

#HTTP-Anfrage senden
response = requests.get(url)
response.raise_for_status()  # Fehler auslösen, falls die Anfrage fehlschlägt

#HTML-Inhalt der Seite parsen
soup = BeautifulSoup(response.text, 'html.parser')

#Tabelle finden
table = soup.find('table', {'id': 'pokedex'})

#Tabellenüberschriften extrahieren
headers = [header.text for header in table.find('thead').find_all('th')]

#Tabelleninhalte extrahieren
rows = []
for row in table.find('tbody').find_all('tr'):
    cells = row.find_all(['th', 'td'])
    rows.append([cell.text.strip() for cell in cells])

#DataFrame erstellen
df = pd.DataFrame(rows, columns=headers)

#Ausgabe des DataFrames
print(df.head())

#Optional: DataFrame in eine CSV-Datei speichern
df.to_csv("pokemon_data.csv", index=False)  

      #                    Name          Type Total  HP Attack Defense  \
0  0001               Bulbasaur  Grass Poison   318  45     49      49   
1  0002                 Ivysaur  Grass Poison   405  60     62      63   
2  0003                Venusaur  Grass Poison   525  80     82      83   
3  0003  Venusaur Mega Venusaur  Grass Poison   625  80    100     123   
4  0004              Charmander          Fire   309  39     52      43   

  Sp. Atk Sp. Def Speed  
0      65      65    45  
1      80      80    60  
2     100     100    80  
3     122     120    80  
4      60      50    65  


In [ ]:
import pandas as pd

# Laden des extrahierten Pokémon-Datensatzes
df = pd.read_csv("pokemon_data.csv")

# Umwandlung von relevanten Spalten in numerische Werte, falls notwendig
numeric_columns = ['#', 'Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']
for col in numeric_columns:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Stärkste Pokémon pro Typ finden
def strongest_pokemon_by_type(df):
    strongest = df.loc[df.groupby('Type')['Total'].idxmax()]
    return strongest[['Name', 'Type', 'Total']]

# Beste Angreifer finden
def best_attackers(df):
    top_attackers = df.sort_values(by='Attack', ascending=False).head(10)
    return top_attackers[['Name', 'Attack']]

# Durchschnittswerte für jeden Typ berechnen
def average_stats_by_type(df):
    averages = df.groupby('Type')[['Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']].mean()
    return averages

# Ergebnisse berechnen
strongest_pokemon = strongest_pokemon_by_type(df)
print("Stärkste Pokémon pro Typ:")
print(strongest_pokemon)

best_attackers = best_attackers(df)
print("\nBeste Angreifer:")
print(best_attackers)

average_stats = average_stats_by_type(df)
print("\nDurchschnittswerte pro Typ:")
print(average_stats)

# Optional: Ergebnisse in Dateien speichern
strongest_pokemon.to_csv("strongest_pokemon_by_type.csv", index=False)
best_attackers.to_csv("best_attackers.csv", index=False)
average_stats.to_csv("average_stats_by_type.csv")
#

Stärkste Pokémon pro Typ:
                          Name           Type  Total
167                     Pinsir            Bug    500
1095                     Lokix       Bug Dark    450
888                   Vikavolt   Bug Electric    500
896                   Ribombee      Bug Fairy    464
274   Heracross Mega Heracross   Bug Fighting    600
...                        ...            ...    ...
176                     Lapras      Water Ice    535
94                  Tentacruel   Water Poison    515
108       Slowbro Mega Slowbro  Water Psychic    590
687                 Carracosta     Water Rock    495
489                   Empoleon    Water Steel    530

[221 rows x 3 columns]

Beste Angreifer:
                         Name  Attack
201      Mewtwo Mega Mewtwo X     190
274  Heracross Mega Heracross     185
956                   Kartana     181
475    Rayquaza Mega Rayquaza     180
478       Deoxys Attack Forme     180
473    Groudon Primal Groudon     180
545    Garchomp Mega Garchomp 